### View Dendrite
Set `dendrite_path` and `scale`.

In [ ]:
from notebook_widgets import show_polylines, show_3d_mesh, polylines_to_line_set
from CGAL.CGAL_Polyhedron_3 import Polyhedron_3
from CGAL.CGAL_Polygon_mesh_processing import Polylines
from CGAL.CGAL_Surface_mesh_skeletonization import surface_mesh_skeletonization
from spine_segmentation import apply_scale

dendrite_path = "example_dendrite/surface_mesh.off"

scale = (1, 1, 1)

dendrite_mesh = Polyhedron_3(dendrite_path)
dendrite_mesh = apply_scale(dendrite_mesh, scale)

# get skeleton
skeleton_polylines = Polylines()
correspondence_polylines = Polylines()
surface_mesh_skeletonization(dendrite_mesh, skeleton_polylines, correspondence_polylines)

print("Mesh")
show_3d_mesh(dendrite_mesh)
print("Skeleton")
show_polylines(skeleton_polylines)
print("Skeleton + Mesh")
show_polylines(skeleton_polylines, dendrite_mesh)

### View Chords
Set `dataset_path`, `scale`, `num_of_chords` and `num_of_bins`.

In [ ]:
from notebook_widgets import SpineMeshDataset, spine_chords_widget
from spine_segmentation import apply_scale


dataset_path = "example_dendrite"

scale = (1, 1, 1)

num_of_chords = 100
num_of_bins = 10

spine_dataset = SpineMeshDataset().load(dataset_path)
spine_dataset.apply_scale(scale)

display(spine_chords_widget(spine_dataset, num_of_chords, num_of_bins))

### View Spines in Dataset
Set `dataset_path` and `scale`.

In [ ]:
from spine_metrics import SpineMetricDataset
from notebook_widgets import SpineMeshDataset, spine_dataset_view_widget

dataset_path = "example_dendrite"

scale = (1, 1, 1)

spine_dataset = SpineMeshDataset().load(dataset_path)
spine_dataset.apply_scale(scale)

metrics = SpineMetricDataset().load(f"{dataset_path}/metrics.csv")

display(spine_dataset_view_widget(spine_dataset, metrics))

### Calculate Metrics
Set `dataset_path`. Metrics for the dataset will be saved to `dataset_path/metrics.csv`. Chords histograms will be saved separately to `dataset_path/chords.csv`. 

In [ ]:
from spine_metrics import SpineMetricDataset
from notebook_widgets import SpineMeshDataset


dataset_path = "example_dendrite"

# load meshes
spine_dataset = SpineMeshDataset().load(dataset_path)

# calculate metrics
metric_names = ["OldChordDistribution", "OpenAngle", "CVD", "AverageDistance",
                "LengthVolumeRatio", "LengthAreaRatio", "JunctionArea",
                "Length", "Area", "Volume", "ConvexHullVolume", "ConvexHullRatio"]
spine_metrics = SpineMetricDataset()
spine_metrics.calculate_metrics(spine_dataset.spine_meshes, metric_names)
spine_metrics.save(f"{dataset_path}/metrics.csv")    

spine_metrics.get_metrics_subset(["OldChordDistribution"]).save_as_array(f"{dataset_path}/chords.csv")